In [1]:
from preprocess import *

data_path = "data/epinions.txt"

preprocess(data_path, 10)

Processed 10000 lines
Processed 20000 lines
Processed 30000 lines
Processed 40000 lines
Processed 50000 lines
Processed 60000 lines
Processed 70000 lines
Processed 80000 lines
Processed 90000 lines
Processed 100000 lines
Processed 110000 lines
Processed 120000 lines
Processed 130000 lines
Processed 140000 lines
Number of users: 1178
Number of items: 15094
Number of interactions: 20084
Sparsity: 0.9988704627008607
Test set size: 1178
    index          user                                               item  \
0      48        200067  Samsung_SGH_N100_Voice_Activated_WAP_Phone_Cel...   
1      66         300ex                      auto_Make-2002_Subaru_Impreza   
2      83      77chelle      Leappad_Microphone_With_Two_Interactive_Books   
3      95   ASourdough4       Hewlett_Packard_LASERJET_PRO_P1606DN_PRINTER   
4     151      AVaddict           pr-Bose_301_Series_V_Main_Stereo_Speaker   
5     161    AgnesVarda         pr-Pentax_UC-1_35mm_Point_and_Shoot_Camera   
6     205       A

In [2]:
from epinions_dataset import EpinionsDataset
import pandas as pd
import pickle

# Get the data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# Get user to items dictionary from pickle file
with open('data/user_to_item.pkl', 'rb') as f:
    user_to_items = pickle.load(f)

# Get items set from pickle file
with open('data/items.pkl', 'rb') as f:
    items_set = pickle.load(f)

# Create dataset
trainset = EpinionsDataset(train, user_to_items, items_set)
print(len(trainset))
print(trainset[0][0].shape)
print(trainset[0][1].shape)



Number of users: 1178
Number of items: 15094
94530
torch.Size([1178])
torch.Size([15094])
